## Setting up your system for this demo

The following code creates two csv files using data from the [Boston Housing dataset](https://archive.ics.uci.edu/ml/datasets/Housing) which is built into scikit-learn and adds them to the local directory

In [1]:
import pandas as pd
import numpy
from numpy.random import choice
from sklearn.datasets import load_boston

import h2o
h2o.init()

H2O cluster uptime:,2 minutes 19 seconds 314 milliseconds
H2O cluster version:,3.7.0.99999
H2O cluster name:,spIdea
H2O cluster total nodes:,1
H2O cluster total free memory:,12.44 GB
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster healthy:,True
H2O Connection ip:,127.0.0.1
H2O Connection port:,54321
H2O Connection proxy:,None


In [2]:
# transfer the boston data from pandas to H2O
boston_data = load_boston()
X = pd.DataFrame(data=boston_data.data, columns=boston_data.feature_names)
X["Median_value"] = boston_data.target
X = h2o.H2OFrame.from_python(X.to_dict("list"))


Parse Progress: [##################################################] 100%


In [3]:
# select 10% for valdation
r = X.runif(seed=123456789)
train = X[r < 0.9,:]
valid = X[r >= 0.9,:]

h2o.export_file(train, "Boston_housing_train.csv", force=True)
h2o.export_file(valid, "Boston_housing_test.csv", force=True)


Export File Progress: [##################################################] 100%

Export File Progress: [##################################################] 100%


Enable inline plotting in the Jupyter Notebook

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt

## Intro to H2O Data Munging

Read csv data into H2O. This loads the data into the H2O column compressed, in-memory, key-value store.

In [5]:
fr = h2o.import_file("Boston_housing_train.csv")


Parse Progress: [##################################################] 100%


View the top of the H2O frame.

In [6]:
fr.head()

RM,ZN,Median_value,TAX,CRIM,NOX,INDUS,LSTAT,RAD,AGE,PTRATIO,CHAS,B,DIS
6.575,18,24,296,0.00632,0.538,2.31,4.98,1,65.2,15.3,0,396.9,4.09
6.421,0,21.6,242,0.02731,0.469,7.07,9.14,2,78.9,17.8,0,396.9,4.9671
7.185,0,34.7,242,0.02729,0.469,7.07,4.03,2,61.1,17.8,0,392.83,4.9671
6.998,0,33.4,222,0.03237,0.458,2.18,2.94,3,45.8,18.7,0,394.63,6.0622
7.147,0,36.2,222,0.06905,0.458,2.18,5.33,3,54.2,18.7,0,396.9,6.0622
6.43,0,28.7,222,0.02985,0.458,2.18,5.21,3,58.7,18.7,0,394.12,6.0622
6.012,12.5,22.9,311,0.08829,0.524,7.87,12.43,5,66.6,15.2,0,395.6,5.5605
6.172,12.5,27.1,311,0.14455,0.524,7.87,19.15,5,96.1,15.2,0,396.9,5.9505
5.631,12.5,16.5,311,0.21124,0.524,7.87,29.93,5,100,15.2,0,386.63,6.0821
6.004,12.5,18.9,311,0.17004,0.524,7.87,17.1,5,85.9,15.2,0,386.71,6.5921


View the bottom of the H2O Frame

In [7]:
fr.tail()

RM,ZN,Median_value,TAX,CRIM,NOX,INDUS,LSTAT,RAD,AGE,PTRATIO,CHAS,B,DIS
5.926,0,24.5,391,0.27957,0.585,9.69,13.59,6,42.6,19.2,0,396.9,2.3817
5.67,0,23.1,391,0.17899,0.585,9.69,17.6,6,28.8,19.2,0,393.29,2.7986
5.794,0,18.3,391,0.26838,0.585,9.69,14.1,6,70.6,19.2,0,396.9,2.8927
6.019,0,21.2,391,0.23912,0.585,9.69,12.92,6,65.3,19.2,0,396.9,2.4091
5.569,0,17.5,391,0.17783,0.585,9.69,15.1,6,73.5,19.2,0,395.77,2.3999
6.027,0,16.8,391,0.22438,0.585,9.69,14.33,6,79.7,19.2,0,396.9,2.4982
6.593,0,22.4,273,0.06263,0.573,11.93,9.67,1,69.1,21,0,391.99,2.4786
6.12,0,20.6,273,0.04527,0.573,11.93,9.08,1,76.7,21,0,396.9,2.2875
6.794,0,22,273,0.10959,0.573,11.93,6.48,1,89.3,21,0,393.45,2.3889
6.03,0,11.9,273,0.04741,0.573,11.93,7.88,1,80.8,21,0,396.9,2.505


Select a column

fr["VAR_NAME"]

In [8]:
fr["CRIM"].head() # Tab completes

CRIM
0.00632
0.02731
0.02729
0.03237
0.06905
0.02985
0.08829
0.14455
0.21124
0.17004


Select a few columns

In [9]:
columns = ["CRIM", "RM", "RAD"]
fr[columns].head()

CRIM,RM,RAD
0.00632,6.575,1
0.02731,6.421,2
0.02729,7.185,2
0.03237,6.998,3
0.06905,7.147,3
0.02985,6.43,3
0.08829,6.012,5
0.14455,6.172,5
0.21124,5.631,5
0.17004,6.004,5


Select a subset of rows

Unlike in Pandas, columns may be identified by index or column name. **Therefore, when subsetting by rows, you must also pass the column selection.**

In [10]:
fr[2:7,:]  # explicitly select all columns with :

RM,ZN,Median_value,TAX,CRIM,NOX,INDUS,LSTAT,RAD,AGE,PTRATIO,CHAS,B,DIS
7.185,0,34.7,242,0.02729,0.469,7.07,4.03,2,61.1,17.8,0,392.83,4.9671
6.998,0,33.4,222,0.03237,0.458,2.18,2.94,3,45.8,18.7,0,394.63,6.0622
7.147,0,36.2,222,0.06905,0.458,2.18,5.33,3,54.2,18.7,0,396.9,6.0622
6.43,0,28.7,222,0.02985,0.458,2.18,5.21,3,58.7,18.7,0,394.12,6.0622
6.012,12.5,22.9,311,0.08829,0.524,7.87,12.43,5,66.6,15.2,0,395.6,5.5605


Key attributes:
      * columns, names, col_names
      * len, shape, dim, nrow, ncol
      * types
      
Note: 

Since the data is _not_ in local python memory
there is no "values" attribute. If you want to 
pull all of the data into the local python memory
then do so explicitly with h2o.export_file and
reading the data into python memory from disk.

In [12]:
# The columns attribute is exactly like Pandas
print("Columns:" + str(fr.columns) + "\n")
print("Columns:" + str(fr.names) + "\n")
print("Columns:" + str(fr.col_names) + "\n")

# There are a number of attributes to get at the shape
print("length:" + str( len(fr) ) + "\n")
print("shape:" + str(fr.shape) + "\n")
print("dim:" + str(fr.dim) + "\n")
print("nrow:" + str(fr.nrow) + "\n")
print("ncol:" + str(fr.ncol) + "\n")

# Use the "types" attribute to list the column types
print("types:" + str(fr.types) + "\n")

Columns:['RM', 'ZN', 'Median_value', 'TAX', 'CRIM', 'NOX', 'INDUS', 'LSTAT', 'RAD', 'AGE', 'PTRATIO', 'CHAS', 'B', 'DIS']

Columns:['RM', 'ZN', 'Median_value', 'TAX', 'CRIM', 'NOX', 'INDUS', 'LSTAT', 'RAD', 'AGE', 'PTRATIO', 'CHAS', 'B', 'DIS']

Columns:['RM', 'ZN', 'Median_value', 'TAX', 'CRIM', 'NOX', 'INDUS', 'LSTAT', 'RAD', 'AGE', 'PTRATIO', 'CHAS', 'B', 'DIS']

length:453

shape:(453, 14)

dim:[453, 14]

nrow:453

ncol:14

types:{'RM': 'real', 'ZN': 'real', 'Median_value': 'real', 'TAX': 'int', 'CRIM': 'real', 'RAD': 'int', 'NOX': 'real', 'INDUS': 'real', 'LSTAT': 'real', 'AGE': 'real', 'PTRATIO': 'real', 'CHAS': 'int', 'B': 'real', 'DIS': 'real'}



Select rows based on value

In [13]:
fr.shape

(453, 14)

Boolean masks can be used to subselect rows based on a criteria.

In [14]:
mask = fr["CRIM"]>1
fr[mask,:].shape

(149, 14)

Get summary statistics of the data and additional data distribution information.

In [15]:
fr.describe()

Rows:453 Cols:14

Chunk compression summary: 


chunk_type,chunk_name,count,count_percentage,size,size_percentage
CBS,Bits,1,7.1428576,127 B,0.3965280
C1N,1-Byte Integers (w/o NAs),1,7.1428576,521 B,1.6267016
C2,2-Byte Integers,1,7.1428576,974 B,3.041089
C2S,2-Byte Fractions,1,7.1428576,990 B,3.0910454
CUD,Unique Reals,4,28.57143,7.1 KB,22.680155
C8D,64-bit Reals,6,42.857143,21.6 KB,69.16448



Frame distribution summary: 


,size,number_of_rows,number_of_chunks_per_column,number_of_chunks
172.16.2.84:54321,31.3 KB,453.0,1.0,14.0
mean,31.3 KB,453.0,1.0,14.0
min,31.3 KB,453.0,1.0,14.0
max,31.3 KB,453.0,1.0,14.0
stddev,0 B,0.0,0.0,0.0
total,31.3 KB,453.0,1.0,14.0


,RM,ZN,Median_value,TAX,CRIM,NOX,INDUS,LSTAT,RAD,AGE,PTRATIO,CHAS,B,DIS
type,real,real,real,int,real,real,real,real,int,real,real,int,real,real
mins,3.561,0.0,5.0,187.0,0.00632,0.385,0.46,1.73,1.0,2.9000000000000004,12.600000000000001,0.0,0.32,1.1296000000000002
mean,6.272730684326713,11.539735099337754,22.528918322295826,402.22737306843294,3.290260088300221,0.5533324503311261,10.931258278145691,12.51885209713025,9.275938189845471,67.77792494481236,18.408830022075065,0.0640176600441501,356.7393818984548,3.839994260485648
maxs,8.78,100.0,50.0,711.0,73.53410000000001,0.871,27.740000000000002,37.97,24.0,100.0,22.0,1.0,396.9,12.1265
sigma,0.7085357660227043,23.31499786818323,9.083731832478104,166.33136494922283,7.830928086015703,0.11725492435439215,6.824765577156836,7.00633158705198,8.563209498577333,27.902760166122622,2.1837820768329164,0.24505502298300838,91.57740504437409,2.095813694508859
zeros,0,330,0,0,0,0,0,0,0,0,0,424,0,0
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,6.575,18.0,24.0,296.0,0.00632,0.538,2.31,4.98,1.0,65.2,15.3,0.0,396.9,4.09
1,6.421,0.0,21.6,242.0,0.02731,0.46900000000000003,7.07,9.14,2.0,78.9,17.8,0.0,396.9,4.9671
2,7.1850000000000005,0.0,34.7,242.0,0.027290000000000002,0.46900000000000003,7.07,4.03,2.0,61.1,17.8,0.0,392.83,4.9671


Set up the predictor and response column names

Using H2O algorithms, it's easier to reference predictor and response columns
by name in a single frame (i.e., don't split up X and y)

In [ ]:
x = fr.names[:]
y="Median_value"
x.remove(y)

## Machine Learning With H2O

H2O is a machine learning library built in Java with interfaces in Python, R, Scala, and Javascript. It is [open source](http://github.com/h2oai) and [well-documented](http://docs.h2o.ai).

Unlike Scikit-learn, H2O allows for categorical and missing data.

The basic work flow is as follows:
* Fit the training data with a machine learning algorithm
* Predict on the testing data

### Simple model

In [ ]:
model = h2o.random_forest(x=fr[:400,x],y=fr[:400,y],seed=42) # Define and fit first 400 points

In [ ]:
model.predict(fr[400:fr.nrow,:])        # Predict the rest

The performance of the model can be checked using the holdout dataset

In [ ]:
perf = model.model_performance(fr[400:fr.nrow,:])
perf.r2()      # get the r2 on the holdout data
perf.mse()     # get the mse on the holdout data
perf           # display the performance object

### Train-Test Split

Instead of taking the first 400 observations for training, we can use H2O to create a random test train split of the data.

In [ ]:
r = fr.runif(seed=12345)   # build random uniform column over [0,1]
train= fr[r<0.75,:]     # perform a 75-25 split
test = fr[r>=0.75,:]

model = h2o.random_forest(x=train[x],y=train[y],seed=42)

perf = model.model_performance(test)
perf.r2()

There was a massive jump in the R^2 value. This is because the original data is not shuffled.

### Cross validation

H2O's machine learning algorithms take an optional parameter **nfolds** to specify the number of cross-validation folds to build. H2O's cross-validation uses an internal weight vector to build the folds in an efficient manner (instead of physically building the splits).

In conjunction with the **nfolds** parameter, a user may specify the way in which observations are assigned to each fold with the **fold_assignment** parameter, which can be set to either:
        * AUTO:  Perform random assignment
        * Random: Each row has a equal (1/nfolds) chance of being in any fold.
        * Modulo: Observations are in/out of the fold based by modding on nfolds

In [ ]:
model = h2o.random_forest(x=fr[x],y=fr[y], nfolds=10) # build a 10-fold cross-validated model

In [ ]:
scores = numpy.array([m.r2() for m in model.xvals]) # iterate over the xval models using the xvals attribute
print("Expected R^2: %.2f +/- %.2f \n" % (scores.mean(), scores.std()*1.96))
print("Scores:" + scores.round(2))

However, you can still make use of the cross_val_score from Scikit-Learn

### Cross validation: H2O and Scikit-Learn

In [ ]:
from sklearn.cross_validation import cross_val_score
from h2o.cross_validation import H2OKFold
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.model.regression import h2o_r2_score
from sklearn.metrics.scorer import make_scorer

You still must use H2O to make the folds. Currently, there is no H2OStratifiedKFold. Additionally, the H2ORandomForestEstimator is analgous to the scikit-learn RandomForestRegressor object with its own ``fit`` method

In [ ]:
model = H2ORandomForestEstimator(seed=42)

In [ ]:
scorer = make_scorer(h2o_r2_score)   # make h2o_r2_score into a scikit_learn scorer
custom_cv = H2OKFold(fr, n_folds=10, seed=42) # make a cv 
scores = cross_val_score(model, fr[x], fr[y], scoring=scorer, cv=custom_cv)

print("Expected R^2: %.2f +/- %.2f \n" % (scores.mean(), scores.std()*1.96))
print("Scores:", scores.round(2))

There isn't much difference in the R^2 value since the fold strategy is exactly the same. However, there was a major difference in terms of computation time and memory usage.

Since the progress bar print out gets annoying let's disable that

In [ ]:
h2o.__PROGRESS_BAR__=False
h2o.no_progress()

### Grid Search

Grid search in H2O is still under active development and it will be available very soon. However, it is possible to make use of Scikit's grid search infrastructure (with some performance penalties)

### Randomized grid search: H2O and Scikit-Learn

In [ ]:
from sklearn import __version__
sklearn_version = __version__
print sklearn_version

If you have 0.16.1, then your system can't handle complex randomized grid searches (it works in every other version of sklearn, including the soon to be released 0.16.2 and the older versions).

The steps to perform a randomized grid search:
1. Import model and RandomizedSearchCV
2. Define model
3. Specify parameters to test
4. Define grid search object
5. Fit data to grid search object
6. Collect scores

All the steps will be repeated from above.

Because 0.16.1 is installed, we use scipy to define specific distributions

ADVANCED TIP:

Turn off reference counting for spawning jobs in parallel (n_jobs=-1, or n_jobs > 1).
We'll turn it back on again in the aftermath of a Parallel job.

If you don't want to run jobs in parallel, don't turn off the reference counting.

Pattern is:
         >>> h2o.turn_off_ref_cnts()
         >>> .... parallel job ....
         >>> h2o.turn_on_ref_cnts()

In [ ]:
%%time
from h2o.estimators.random_forest import H2ORandomForestEstimator  # Import model
from sklearn.grid_search import RandomizedSearchCV  # Import grid search
from scipy.stats import randint, uniform

model = H2ORandomForestEstimator(seed=42)        # Define model

params = {"ntrees": randint(20,50),
          "max_depth": randint(1,10),
          "min_rows": randint(1,10),    # scikit's  min_samples_leaf
          "mtries": randint(2,fr[x].shape[1]),} # Specify parameters to test

scorer = make_scorer(h2o_r2_score)   # make h2o_r2_score into a scikit_learn scorer
custom_cv = H2OKFold(fr, n_folds=10, seed=42) # make a cv 
random_search = RandomizedSearchCV(model, params, 
                                   n_iter=30, 
                                   scoring=scorer, 
                                   cv=custom_cv, 
                                   random_state=42,
                                   n_jobs=1)       # Define grid search object

random_search.fit(fr[x], fr[y])

print("Best R^2:" + random_search.best_score_ + "\n")
print("Best params:" + random_search.best_params_)

We might be tempted to think that we just had a large improvement; however we must be cautious. The function below creates a more detailed report.

In [ ]:
def report_grid_score_detail(random_search, charts=True):
    """Input fit grid search estimator. Returns df of scores with details"""
    df_list = []

    for line in random_search.grid_scores_:
        results_dict = dict(line.parameters)
        results_dict["score"] = line.mean_validation_score
        results_dict["std"] = line.cv_validation_scores.std()*1.96
        df_list.append(results_dict)

    result_df = pd.DataFrame(df_list)
    result_df = result_df.sort("score", ascending=False)
    
    if charts:
        for col in get_numeric(result_df):
            if col not in ["score", "std"]:
                plt.scatter(result_df[col], result_df.score)
                plt.title(col)
                plt.show()

        for col in list(result_df.columns[result_df.dtypes == "object"]):
            cat_plot = result_df.score.groupby(result_df[col]).mean()[0]
            cat_plot.sort()
            cat_plot.plot(kind="barh", xlim=(.5, None), figsize=(7, cat_plot.shape[0]/2))
            plt.show()
    return result_df

def get_numeric(X):
    """Return list of numeric dtypes variables"""
    return X.dtypes[X.dtypes.apply(lambda x: str(x).startswith(("float", "int", "bool")))].index.tolist()

In [ ]:
report_grid_score_detail(random_search).head()

Based on the grid search report, we can narrow the parameters to search and rerun the analysis. The parameters below were chosen after a few runs:

In [ ]:
%%time

params = {"ntrees": randint(30,40),
          "max_depth": randint(4,10),
          "mtries": randint(4,10),}

custom_cv = H2OKFold(fr, n_folds=5, seed=42)           # In small datasets, the fold size can have a big
                                                       # impact on the std of the resulting scores. More
random_search = RandomizedSearchCV(model, params,      # folds --> Less examples per fold --> higher 
                                   n_iter=10,          # variation per sample
                                   scoring=scorer, 
                                   cv=custom_cv, 
                                   random_state=43, 
                                   n_jobs=1)       

random_search.fit(fr[x], fr[y])

print("Best R^2:" + random_search.best_score_ + "\n")
print("Best params:" + random_search.best_params_)

report_grid_score_detail(random_search)

### Transformations

Rule of machine learning: Don't use your testing data to inform your training data. Unfortunately, this happens all the time when preparing a dataset for the final model. But on smaller datasets, you must be especially careful.

At the moment, there are no classes for managing data transformations. On the one hand, this requires the user to tote around some extra state, but on the other, it allows the user to be more explicit about transforming H2OFrames.

Basic steps:

0. Remove the response variable from transformations.
1. Import transformer
2. Define transformer
3. Fit train data to transformer
4. Transform test and train data
5. Re-attach the response variable.

First let's normalize the data using the means and standard deviations of the training data.
Then let's perform a principal component analysis on the training data and select the top 5 components.
Using these components, let's use them to reduce the train and test design matrices.

In [ ]:
from h2o.transforms.preprocessing import H2OScaler
from h2o.transforms.decomposition import H2OPCA

#### Normalize Data: Use the means and standard deviations from the training data.

In [ ]:
y_train = train.pop("Median_value")
y_test  = test.pop("Median_value")

In [ ]:
norm = H2OScaler()
norm.fit(train)
X_train_norm = norm.transform(train)
X_test_norm  = norm.transform(test)

In [ ]:
print(X_test_norm.shape)
X_test_norm

Then, we can apply PCA and keep the top 5 components.

In [ ]:
pca = H2OPCA(k=5)
pca.fit(X_train_norm)
X_train_norm_pca = pca.transform(X_train_norm)
X_test_norm_pca  = pca.transform(X_test_norm)

In [ ]:
# prop of variance explained by top 5 components?

In [ ]:
print(X_test_norm_pca.shape)
X_test_norm_pca[:5]

In [ ]:
model = H2ORandomForestEstimator(seed=42)
model.fit(X_train_norm_pca,y_train)
y_hat  = model.predict(X_test_norm_pca)

In [ ]:
h2o_r2_score(y_test,y_hat)

Although this is MUCH simpler than keeping track of all of these transformations manually, it gets to be somewhat of a burden when you want to chain together multiple transformers.

### Pipelines

"Tranformers unite!"

If your raw data is a mess and you have to perform several transformations before using it, use a pipeline to keep things simple.

Steps:

1. Import Pipeline, transformers, and model
2. Define pipeline. The first and only argument is a *list* of *tuples* where the first element of each tuple is a name you give the step and the second element is a defined transformer. The last step is optionally an estimator class (like a RandomForest).
3. Fit the training data to pipeline
4. Either transform or predict the testing data

In [ ]:
from h2o.transforms.preprocessing import H2OScaler
from h2o.transforms.decomposition import H2OPCA
from h2o.estimators.random_forest import H2ORandomForestEstimator

In [ ]:
from sklearn.pipeline import Pipeline                # Import Pipeline <other imports not shown>
model = H2ORandomForestEstimator(seed=42)
pipe = Pipeline([("standardize", H2OScaler()),       # Define pipeline as a series of steps
                 ("pca", H2OPCA(k=5)),
                 ("rf", model)])                     # Notice the last step is an estimator

pipe.fit(train, y_train)                             # Fit training data
y_hat = pipe.predict(test)                           # Predict testing data (due to last step being an estimator)
h2o_r2_score(y_test, y_hat)                          # Notice the final score is identical to before

This is so much easier!!!

But, wait a second, we did worse after applying these transformations! We might wonder how different hyperparameters for the transformations impact the final score.

### Combining randomized grid search and pipelines
"Yo dawg, I heard you like models, so I put models in your models to model models."

Steps:

1. Import Pipeline, grid search, transformers, and estimators <Not shown below>
2. Define pipeline
3. Define parameters to test in the form: "(Step name)__(argument name)" A double underscore separates the two words.
4. Define grid search
5. Fit to grid search

In [ ]:
pipe = Pipeline([("standardize", H2OScaler()),
                 ("pca", H2OPCA()),
                 ("rf", H2ORandomForestEstimator(seed=42))])

params = {"standardize__center":    [True, False],           # Parameters to test
          "standardize__scale":     [True, False],
          "pca__k":                 randint(2, 6),
          "rf__ntrees":             randint(50,80),
          "rf__max_depth":          randint(4,10),
          "rf__min_rows":           randint(5,10), }
#           "rf__mtries":             randint(1,4),}           # gridding over mtries is 
                                                               # problematic with pca grid over 
                                                               # k above 

from sklearn.grid_search import RandomizedSearchCV
from h2o.cross_validation import H2OKFold
from h2o.model.regression import h2o_r2_score
from sklearn.metrics.scorer import make_scorer

custom_cv = H2OKFold(fr, n_folds=5, seed=42)
random_search = RandomizedSearchCV(pipe, params,
                                   n_iter=30,
                                   scoring=make_scorer(h2o_r2_score),
                                   cv=custom_cv,
                                   random_state=42,
                                   n_jobs=1)


random_search.fit(fr[x],fr[y])
results = report_grid_score_detail(random_search)
results.head()

Currently Under Development (drop-in scikit-learn pieces):
    * Richer set of transforms (only PCA and Scale are implemented)
    * Richer set of estimators (only RandomForest is available)
    * Full H2O Grid Search

### Other Tips: Model Save/Load

It is useful to save constructed models to disk and reload them between H2O sessions. Here's how:

In [ ]:
best_estimator = random_search.best_estimator_                        # fetch the pipeline from the grid search
h2o_model      = h2o.get_model(best_estimator._final_estimator._id)    # fetch the model from the pipeline

In [ ]:
save_path = h2o.save_model(h2o_model, path=".", force=True)
print(save_path)

In [ ]:
# assumes new session
my_model = h2o.load_model(path=save_path)

In [ ]:
my_model.predict(X_test_norm_pca)